In [1]:
#all data wrangling

#imports
import os
import glob
import pandas as pd
import random
import numpy as np
from datetime import datetime
import csv
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,DBSCAN
from sklearn.preprocessing import StandardScaler, Normalizer

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals.six import StringIO
import subprocess

#file folder
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#rename columns
combined_csv.columns = combined_csv.columns.map(lambda x: x.replace(" ","_"))
combined_csv.columns = combined_csv.columns.map(lambda x: x.replace("\n",""))
#standardize neighborhood tags
combined_csv['NEIGHBORHOOD'] =  combined_csv['NEIGHBORHOOD'].replace(' ', '', regex=True)
#grosss square feet
combined_csv['GROSS_SQUARE_FEET'] =  combined_csv['GROSS_SQUARE_FEET'].replace(' ', '', regex=True).replace('-', '', regex=True).replace(',', '', regex=True)
combined_csv['GROSS_SQUARE_FEET'].replace('', np.nan, inplace=True)
combined_csv.dropna(subset=['GROSS_SQUARE_FEET'])
combined_csv['GROSS_SQUARE_FEET'] =  combined_csv['GROSS_SQUARE_FEET'].astype(float)
#sale price
combined_csv['SALE_PRICE'] =  combined_csv['SALE_PRICE'].replace(' ', '', regex=True).replace('-', '', regex=True).replace(',', '', regex=True).replace('[\$,]', '', regex=True)
combined_csv['SALE_PRICE'].replace('', np.nan, inplace=True)
combined_csv.dropna(subset=['SALE_PRICE'])
combined_csv['SALE_PRICE'] =  combined_csv['SALE_PRICE'].astype(float)
#residential
combined_csv['RESIDENTIAL_UNITS'] =  combined_csv['RESIDENTIAL_UNITS'].replace(',', '', regex=True).replace('-', '', regex=True).replace(' ', '', regex=True)
combined_csv['RESIDENTIAL_UNITS'].replace('', np.nan, inplace=True)
combined_csv['RESIDENTIAL_UNITS'] =  combined_csv['RESIDENTIAL_UNITS'].astype(float)
#commercial
combined_csv['COMMERCIAL_UNITS'] =  combined_csv['COMMERCIAL_UNITS'].replace(',', '', regex=True).replace('-', '', regex=True).replace(' ', '', regex=True)
combined_csv['COMMERCIAL_UNITS'].replace('', np.nan, inplace=True)
combined_csv['COMMERCIAL_UNITS'].fillna(0, inplace=True)
combined_csv['COMMERCIAL_UNITS'] =  combined_csv['COMMERCIAL_UNITS'].astype(float)
#yearbuilt 
combined_csv['YEAR_BUILT'] =  combined_csv['YEAR_BUILT'].replace(',', '', regex=True).replace('-', '', regex=True).replace(' ', '', regex=True)
combined_csv['YEAR_BUILT'].replace('', np.nan, inplace=True)
combined_csv['YEAR_BUILT'] =  combined_csv['YEAR_BUILT'].astype(float)
#create year column
combined_csv['SALE_DATEadj'] = pd.to_datetime(combined_csv['SALE_DATE'])
combined_csv['SALE_YEAR'] = combined_csv['SALE_DATEadj'].dt.year
#counter
combined_csv['counter'] = 1

combined_csv.dropna(subset=['ZIP_CODE'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,ADDRESS,APARTMENT_NUMBER,BLOCK,BOROUGH,BUILDING_CLASS_AS_OF_FINAL_ROLL_17/18,BUILDING_CLASS_AS_OF_FINAL_ROLL_18/19,BUILDING_CLASS_AT_PRESENT,BUILDING_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_CATEGORY,COMMERCIAL_UNITS,...,TAX_CLASS_AS_OF_FINAL_ROLL_17/18,TAX_CLASS_AS_OF_FINAL_ROLL_18/19,TAX_CLASS_AT_PRESENT,TAX_CLASS_AT_TIME_OF_SALE,TOTAL_UNITS,YEAR_BUILT,ZIP_CODE,SALE_DATEadj,SALE_YEAR,counter
0,96 AVENUE B,,402,1,NaN,NaN,S1,S1,01 ONE FAMILY DWELLINGS,1.0,...,NaN,NaN,1,1,2,1901.0,10009,2014-12-05,2014,1
1,17 AVENUE D,,372,1,NaN,NaN,S2,S2,02 TWO FAMILY DWELLINGS,1.0,...,NaN,NaN,1,1,3,1910.0,10002,2014-01-31,2014,1
2,243 EAST 7TH STREET,,377,1,NaN,NaN,C0,C0,03 THREE FAMILY DWELLINGS,0.0,...,NaN,NaN,1,1,3,1899.0,10009,2014-11-26,2014,1
3,300 EAST 2ND STREET,,372,1,NaN,NaN,C7,C7,07 RENTALS - WALKUP APARTMENTS,2.0,...,NaN,NaN,2B,2,10,1920.0,10009,2014-07-18,2014,1
4,324 EAST 4TH STREET,,373,1,NaN,NaN,D1,C3,07 RENTALS - WALKUP APARTMENTS,0.0,...,NaN,NaN,2,2,11,1920.0,10009,2014-05-30,2014,1
5,285 EAST 3RD STREET,,373,1,NaN,NaN,C3,C3,07 RENTALS - WALKUP APARTMENTS,0.0,...,NaN,NaN,2A,2,4,1900.0,10009,2014-08-05,2014,1
6,273 EAST 7TH STREET,,377,1,NaN,NaN,C5,C5,07 RENTALS - WALKUP APARTMENTS,0.0,...,NaN,NaN,2B,2,8,1900.0,10009,2014-10-16,2014,1
7,271 EAST 7TH STREET,,377,1,NaN,NaN,C2,C2,07 RENTALS - WALKUP APARTMENTS,0.0,...,NaN,NaN,2A,2,5,1900.0,10009,2014-11-03,2014,1
8,271 EAST 7TH STREET,,377,1,NaN,NaN,C2,C2,07 RENTALS - WALKUP APARTMENTS,0.0,...,NaN,NaN,2A,2,5,1900.0,10009,2014-07-30,2014,1
9,271 EAST 7TH STREET,,377,1,NaN,NaN,C2,C2,07 RENTALS - WALKUP APARTMENTS,0.0,...,NaN,NaN,2A,2,5,1900.0,10009,2014-04-28,2014,1


In [2]:
#new building purchases
combined_csv['SALE_YEAR'] =  combined_csv['SALE_YEAR'].astype(float)

combined_csv['NEW'] = combined_csv['YEAR_BUILT'] - combined_csv['SALE_YEAR']

combined_csv['NEW'] = combined_csv['NEW'].abs() < 3

In [3]:
#add decade of year built
minYear=1870
maxYear=2020

minDecade = np.floor_divide(minYear,10)*10 #5
maxDecade = np.floor_divide(maxYear,10)*10 #6

allDecades = np.arange(minDecade,maxDecade+10,10)

combined_csv["DECADE_BUILT"] = pd.cut(combined_csv.YEAR_BUILT,allDecades) #10

In [4]:
#exports scrubbed data

combined_csv.to_csv('data\REdata.csv', sep='*')